In [190]:
import os
import sys
import pandas as pd
import matplotlib.pyplot as plt 
import numpy as np

In [237]:
folder_name = '/Users/andrei-macpro/Documents/Data/openpose/play/tracking/tracking/1080_play'
# do it for one folder first
# load each file in a pandas dataframe

tracks = []
for filename in sorted(os.listdir(folder_name)):
        # load the file in a pandas dataframe
    if filename == ".DS_Store":
        continue    
    print(filename)
    filepath = os.path.join(folder_name, filename)
    df = pd.read_csv(filepath, index_col=0)
    # Remove rows with duplicated index values because the interpolation sometimes created duplicates
    df = df[~df.index.duplicated(keep='first')]
    df = clean_outliers(df)
    tracks.append(df)


cg.csv
child.csv


In [238]:
cg = tracks[0]
child = tracks[1]

In [239]:
# Check if cg has duplicate indices
if cg.index.duplicated().any():
    print("cg has duplicate indices")

# Check if child has duplicate indices
if child.index.duplicated().any():
    print("child has duplicate indices")

In [236]:
duplicate_indices_child = child[child.index.duplicated(keep=False)]
print(duplicate_indices_child)

                x           y
frame                        
218.0  414.201730  263.267873
219.0  415.541364  262.803021
220.0  416.880997  262.338168
221.0  418.220630  261.873315
222.0  419.560263  261.408462
...           ...         ...
699.0  471.968621  285.402293
693.0  471.320806  278.268158
694.0  471.502628  279.925443
695.0  471.748997  283.437443
696.0  471.975031  288.518989

[236 rows x 2 columns]


In [232]:
combined = pd.concat([cg, child], axis=1)

InvalidIndexError: Reindexing only valid with uniquely valued Index objects

In [194]:
combined


,x,y,x,y
frame,,,,
187.0,200.290909,216.000000,258.981541,317.839450
188.0,202.597839,216.000000,259.997889,321.504774
189.0,204.753303,215.546853,260.154626,322.936780
190.0,206.505327,216.508074,259.611188,322.935420
191.0,207.758366,216.671918,258.799914,322.810655
...,...,...,...,...
7857.0,NaN,NaN,474.780874,268.461257
7941.0,NaN,NaN,474.728708,267.356335
7945.0,NaN,NaN,475.384219,272.410245


In [195]:
# select first column from df 
df.iloc[:, 0]

frame
1.0       180.000000
2.0       259.804196
186.0     257.918373
187.0     258.981541
188.0     259.997889
             ...    
8712.0    295.207028
8713.0    295.207337
8714.0    295.205468
8715.0    295.201304
8716.0    295.194729
Name: x, Length: 8179, dtype: float64

In [196]:
# Calculate z-scores for x and y columns
# make this code into a function that takes a dataframe and returns the cleaned dataframe

from scipy.stats import zscore
def clean_outliers(df):
    z_scores_x = zscore(df.iloc[:,0])
    z_scores_y = zscore(df.iloc[:,1])

    # Define threshold for z-score (e.g., 3)
    threshold = 3

    # Identify outliers based on z-scores
    outliers_x = df[abs(z_scores_x) > threshold]
    outliers_y = df[abs(z_scores_y) > threshold]

    # Remove outliers from DataFrame
    cleaned_df = df[~df.index.isin(outliers_x.index) & ~df.index.isin(outliers_y.index)]
    return cleaned_df

In [197]:
combined_scaled[len(child):]

array([], shape=(0, 4), dtype=float64)

In [198]:
from sklearn.preprocessing import StandardScaler
# Combine both datasets for joint scaling

scaler = StandardScaler()
combined_scaled = scaler.fit_transform(combined_data)

# Split the scaled data back into caregiver and child data
scaled_caregiver = combined_scaled[:len(cg)]
scaled_child = combined_scaled[len(cg):]

cg[['x', 'y']] = scaled_caregiver
child[['x', 'y']] = scaled_child




ValueError: Columns must be same length as key

In [ ]:
cg

,x,y
frame,,
187.0,-0.928284,-0.287692
188.0,-0.905541,-0.287692
189.0,-0.884291,-0.298031
190.0,-0.867018,-0.276100
191.0,-0.854665,-0.272361
...,...,...
8712.0,-1.128325,-0.288623
8713.0,-1.128325,-0.288752
8714.0,-1.128325,-0.288521


In [ ]:
aligned_positions = pd.merge(child, cg, on='frame', suffixes=('_caregiver', '_child'))
print("Aligned Positions Data:")
print(aligned_positions)

Aligned Positions Data:
        x_caregiver  y_caregiver   x_child   y_child
frame                                               
187.0     -0.349673     2.035855 -0.928284 -0.287692
188.0     -0.339653     2.119482 -0.905541 -0.287692
189.0     -0.338108     2.152155 -0.884291 -0.298031
190.0     -0.343465     2.152124 -0.867018 -0.276100
191.0     -0.351463     2.149277 -0.854665 -0.272361
...             ...          ...       ...       ...
8712.0     0.007462     3.816670 -1.128325 -0.288623
8713.0     0.007465     3.872428 -1.128325 -0.288752
8714.0     0.007447     3.940504 -1.128325 -0.288521
8715.0     0.007406     4.025797 -1.128325 -0.287871
8716.0     0.007341     4.133203 -1.128325 -0.286744

[8157 rows x 4 columns]


In [ ]:
aligned_positions.index

Float64Index([ 187.0,  188.0,  189.0,  190.0,  191.0,  192.0,  193.0,  194.0,
               195.0,  196.0,
              ...
              8707.0, 8708.0, 8709.0, 8710.0, 8711.0, 8712.0, 8713.0, 8714.0,
              8715.0, 8716.0],
             dtype='float64', name='frame', length=8157)

In [ ]:
distances = []
frame_indices = aligned_positions.index
for i in range(len(aligned_positions) - 1):
    if frame_indices[i + 1] - frame_indices[i] < 5:
        distance = np.sqrt((aligned_positions['x_caregiver'].iloc[i] - aligned_positions['x_child'].iloc[i])**2 + 
                           (aligned_positions['y_caregiver'].iloc[i] - aligned_positions['y_child'].iloc[i])**2)
        distances.append(distance)
    else:
        distances.append(np.nan)  # Or some placeholder for frames that don't meet the criteria

distances


[2.3945062397881567,
 2.472795046830484,
 2.5103233141245096,
 2.48402397215404,
 2.4733668645246145,
 2.4296579898153996,
 2.4881619795019563,
 2.496476489117927,
 2.4729570605751796,
 2.4798815949848807,
 2.462467505081608,
 2.434538515572976,
 2.4068379820556056,
 2.3917808508101137,
 2.3891043355402077,
 2.4122201182861795,
 2.4291368585890596,
 2.433969291244715,
 2.4685226502184636,
 2.515705839775206,
 2.5513209335947185,
 2.5554741724439474,
 2.5229560049281683,
 2.5033934606348,
 2.4572738428824685,
 2.4057987227220283,
 2.323755444734556,
 2.274556455465982,
 2.2667816541362455,
 2.308341151263879,
 2.3408387525062326,
 2.345570198320217,
 2.331675296246022,
 2.3227219963058907,
 2.2946674939591882,
 2.284356945209007,
 2.306253468589073,
 2.3379847808025946,
 2.3519947348423167,
 2.392355936241052,
 2.3673845104396967,
 2.3322247753096494,
 2.3066148292789994,
 2.284085004910905,
 2.308273756894026,
 2.2873112772260646,
 2.2624968058650703,
 2.2810200130036486,
 2.2992504504

In [ ]:
distances

[2.3945062397881567,
 2.472795046830484,
 2.5103233141245096,
 2.48402397215404,
 2.4733668645246145,
 2.4296579898153996,
 2.4881619795019563,
 2.496476489117927,
 2.4729570605751796,
 2.4798815949848807,
 2.462467505081608,
 2.434538515572976,
 2.4068379820556056,
 2.3917808508101137,
 2.3891043355402077,
 2.4122201182861795,
 2.4291368585890596,
 2.433969291244715,
 2.4685226502184636,
 2.515705839775206,
 2.5513209335947185,
 2.5554741724439474,
 2.5229560049281683,
 2.5033934606348,
 2.4572738428824685,
 2.4057987227220283,
 2.323755444734556,
 2.274556455465982,
 2.2667816541362455,
 2.308341151263879,
 2.3408387525062326,
 2.345570198320217,
 2.331675296246022,
 2.3227219963058907,
 2.2946674939591882,
 2.284356945209007,
 2.306253468589073,
 2.3379847808025946,
 2.3519947348423167,
 2.392355936241052,
 2.3673845104396967,
 2.3322247753096494,
 2.3066148292789994,
 2.284085004910905,
 2.308273756894026,
 2.2873112772260646,
 2.2624968058650703,
 2.2810200130036486,
 2.2992504504

In [ ]:
# ok but what if there are actually more gaps than just the one ? how do we keep both the aligned positions and the distances in sync?
distances

[2.3945062397881567,
 2.472795046830484,
 2.5103233141245096,
 2.48402397215404,
 2.4733668645246145,
 2.4296579898153996,
 2.4881619795019563,
 2.496476489117927,
 2.4729570605751796,
 2.4798815949848807,
 2.462467505081608,
 2.434538515572976,
 2.4068379820556056,
 2.3917808508101137,
 2.3891043355402077,
 2.4122201182861795,
 2.4291368585890596,
 2.433969291244715,
 2.4685226502184636,
 2.515705839775206,
 2.5513209335947185,
 2.5554741724439474,
 2.5229560049281683,
 2.5033934606348,
 2.4572738428824685,
 2.4057987227220283,
 2.323755444734556,
 2.274556455465982,
 2.2667816541362455,
 2.308341151263879,
 2.3408387525062326,
 2.345570198320217,
 2.331675296246022,
 2.3227219963058907,
 2.2946674939591882,
 2.284356945209007,
 2.306253468589073,
 2.3379847808025946,
 2.3519947348423167,
 2.392355936241052,
 2.3673845104396967,
 2.3322247753096494,
 2.3066148292789994,
 2.284085004910905,
 2.308273756894026,
 2.2873112772260646,
 2.2624968058650703,
 2.2810200130036486,
 2.2992504504

In [205]:
# do it for all now 
from moviepy.editor import VideoFileClip
f_name = '/Users/andrei-macpro/Documents/Data/videos/play_videos' 

durations = []
file_names = []

for folder_name in sorted(os.listdir(f_name)):
    if folder_name == ".DS_Store":
        continue
    file_path = os.path.join(f_name, folder_name)
    if os.path.isfile(file_path):
        clip = VideoFileClip(file_path)
        durations.append(clip.duration)
        file_names.append(folder_name.split('.')[0])

durations = pd.DataFrame({'file_name': file_names, 'duration': durations})


In [206]:
durations

,file_name,duration
0,1043_play,100.32
1,1043_play_1,320.88
2,1047_play,68.24
3,1047_play_1,108.96
4,1047_play_2,71.73
...,...,...
76,2009_play,638.56
77,2025_play_1,791.02
78,2025_play_2,208.53
79,2027_play,255.10


In [204]:
# how to select a row by its cell name in a dataframe
df.loc[df['file_name'] == '1043_play']['duration'].values[0]

100.32

In [ ]:
# save df to csv in the folder /Users/andrei-macpro/Documents/Data/videos/durations
df.to_csv('/Users/andrei-macpro/Documents/Data/videos/durations/durations_play.csv')

In [300]:
f_name = '/Users/andrei-macpro/Documents/Data/openpose/play/tracking/tracking' 

In [301]:
# now do it for all videos
stats = {}
for folder_name in sorted(os.listdir(f_name)):
    if folder_name == ".DS_Store":
        continue
    file_path = os.path.join(f_name, folder_name)
    tracks = {}
    for file in sorted(os.listdir(file_path)):
                # load the file in a pandas dataframe
            if file == ".DS_Store":
                continue    
            filepath = os.path.join(f_name,folder_name, file)
            df = pd.read_csv(filepath, index_col=0)
            df = df[~df.index.duplicated(keep='first')]
            df.columns = ['x_' + file.split('.')[0], 'y_' + file.split('.')[0]]
            df = clean_outliers(df)
            tracks[file.split('.')[0]] = df

    print(folder_name)
    combined_data = pd.concat([df for df in tracks.values()], axis=1)
    combined_scaled = scaler.fit_transform(combined_data)
    # Convert the scaled data back to a DataFrame
    combined_scaled_df = pd.DataFrame(combined_scaled, index=combined_data.index, columns=combined_data.columns)

# Split the combined_scaled_df back into separate DataFrames and assign them back to the tracks dictionary
    for track_name in tracks.keys():
        x_cols = [col for col in combined_scaled_df.columns if col.startswith('x_' + track_name)]
        y_cols = [col for col in combined_scaled_df.columns if col.startswith('y_' + track_name)]
        tracks[track_name] = combined_scaled_df[x_cols + y_cols]

    # Calculate Euclidean distances
    for track_name, track_df in tracks.items():
        distances = []
        frame_indices = track_df.index
        
        for i in range(len(track_df)-1):
            
            if frame_indices[i + 1] - frame_indices[i] < 5:
                distance = np.sqrt((track_df['x_' + track_name].iloc[i] - track_df['x_' + track_name].iloc[i+1])**2 + 
                            (track_df['y_' + track_name].iloc[i] - track_df['y_' + track_name].iloc[i+1])**2)
                distances.append(distance)
            else:
                distances.append(np.nan)
        
         # Append np.nan to match the number of rows in the DataFrame
        distances.append(np.nan)
        track_df['distance'] = distances

        duration = durations.loc[durations['file_name'] == folder_name]['duration'].values[0]
        mean_movement = track_df['distance'].mean() / duration*100
        median_movement = track_df['distance'].median() / duration*100
        var_movement = track_df['distance'].var() / duration*100
        std_movement = track_df['distance'].std() / duration*100
        min_movement = track_df['distance'].min() / duration*100
        max_movement =  track_df['distance'].max() / duration*100
        skewness_movement = track_df['distance'].skew() / duration*100
        kurtosis_movement = track_df['distance'].kurtosis() / duration*100

        
        if folder_name not in stats:
            stats[folder_name] = {}
        stats[folder_name][track_name] = {'movement_mean': mean_movement, 'movement_var': var_movement, 'movement_std': std_movement, 'movement_min': min_movement, 'movement_max': max_movement, 'movement_skew': skewness_movement, 'movement_kurt': kurtosis_movement}

# normalize the distances (mean, variance etc) by the duration of the video
    

1043_play
1043_play_1
1047_play
1047_play_1
1047_play_2
1049_play
1053_play
1053_play_1
1059_play
1059_play_1
1062_play
1062_play_1
1069_play
1073_play
1073_play_1
1079_play
1080_play
1082_play
1089_play
1091_play
1092_play
1092_play_1
1093_play
1096_play
1096_play_1
1097_play
1098_play
1098_play_1
1099_play
1104_play
1104_play_1
1105_play
1107_play
1108_play
1112_play
1117_play
1117_play_1
1118_play
1122_play
1122_play_1
1124_play
1124_play_1
1125_play
1129_play
1130_play
1130_play_1
1131_play
1132_play
1148_play
1161_play
1170_play
1180_play
1181_play
1184_play
1184_play_1
1186_play
1188_play
1190_play
1195_play
1206_play
1206_play_1
1210_play
1217_play
1230_play
1234_play
1234_play_1
1241_play
1241_play_1
1241_play_2
1245_play
1246_play
1246_play_1
1250_play
1263_play
1282_play
2009_play
2025_play_1
2025_play_2
2027_play
2027_play_1


In [302]:
# Convert the stats dictionary to a DataFrame
df = pd.DataFrame.from_dict({(i,j): stats[i][j] 
                           for i in stats.keys() 
                           for j in stats[i].keys()},
                           orient='index')

# Set the index names
df.index.names = ['s_id', 'track_name']

In [257]:
df

movement_mean  movement_var  movement_std  \
s_id        track_name                                              
1043_play   cg               0.082419      0.020488      0.142908   
            child            0.165271      0.026039      0.161108   
1043_play_1 cg               0.010875      0.000401      0.011184   
            child            0.019620      0.000802      0.015813   
1047_play   cg               0.133647      0.010254      0.122581   
...                               ...           ...           ...   
2025_play_2 child            0.015568      0.000603      0.017006   
2027_play   cg               0.015706      0.002750      0.032832   
            child            0.036506      0.015312      0.077476   
2027_play_1 cg               0.082598      0.010070      0.119768   
            child            0.213759      0.045764      0.255325   

                        movement_min  movement_max  movement_skew  \
s_id        track_name                                              
1043_play   cg          2.268697e-15      1.085276       2.774289   
            child       0.000000e+00      1.140527       1.215600   
1043_play_1 cg          0.000000e+00      0.105434       0.772167   
            child       3.641704e-08      0.136891       0.490042   
1047_play   cg          8.972567e-13      1.462237       3.482644   
...                              ...           ...            ...   
2025_play_2 child       0.000000e+00      0.184164       1.524751   
2027_play   cg          0.000000e+00      1.226659       6.909978   
            child       0.000000e+00      1.160828       3.131730   
2027_play_1 cg          0.000000e+00      1.031517       3.476709   
            child       0.000000e+00      6.286721      12.453608   

                        movement_kurt  
s_id        track_name                 
1043_play   cg               9.768430  
            child            2.083260  
1043_play_1 cg               3.163124  
            child            1.347188  
1047_play   cg              20.589143  
...                               ...  
2025_play_2 child            8.214690  
2027_play   cg             202.136431  
            child           30.897358  
2027_play_1 cg              13.331876  
            child          272.723490  

[160 rows x 7 columns]

In [303]:
# Reset only the second level of the index (track_name)
df.reset_index(level=1, inplace=True)

# Create new DataFrame to hold results
new_df = pd.DataFrame(index=df.index.unique())

# Loop over each unique track_name
for track_name in df['track_name'].unique():
    # Select rows for this track_name
    temp_df = df[df['track_name'] == track_name].copy()
    # Drop the 'track_name' column as it's no longer needed
    temp_df.drop(columns=['track_name'], inplace=True)
    # Add the track_name as a prefix to each column name
    temp_df.columns = [f'{track_name}_{col}' for col in temp_df.columns]
    # Add the results to new_df
    new_df = new_df.join(temp_df)

new_df

,cg_movement_mean,cg_movement_var,cg_movement_std,cg_movement_min,cg_movement_max,cg_movement_skew,cg_movement_kurt,child_movement_mean,child_movement_var,child_movement_std,child_movement_min,child_movement_max,child_movement_skew,child_movement_kurt
s_id,,,,,,,,,,,,,,
1043_play,0.082419,0.020488,0.142908,2.268697e-15,1.085276,2.774289,9.768430,0.165271,0.026039,0.161108,0.000000e+00,1.140527,1.215600,2.083260
1043_play_1,0.010875,0.000401,0.011184,0.000000e+00,0.105434,0.772167,3.163124,0.019620,0.000802,0.015813,3.641704e-08,0.136891,0.490042,1.347188
1047_play,0.133647,0.010254,0.122581,8.972567e-13,1.462237,3.482644,20.589143,0.169502,0.012603,0.135898,3.649836e-05,1.247023,2.458823,9.084948
1047_play_1,0.039350,0.001660,0.039029,3.514601e-12,0.325543,2.090927,7.072152,0.054137,0.002418,0.047111,1.975792e-07,0.464613,2.326316,12.083654
1047_play_2,0.055172,0.001378,0.043832,8.903672e-09,0.306218,2.565166,6.926518,0.117250,0.004347,0.077844,5.888341e-05,0.483883,1.359596,1.884371
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2009_play,0.009420,0.001085,0.013036,0.000000e+00,0.397577,0.808860,14.924153,0.015301,0.001673,0.016188,0.000000e+00,0.526478,0.951577,21.554580
2025_play_1,0.004570,0.000304,0.006201,0.000000e+00,0.068716,0.201123,0.368466,0.004500,0.000193,0.004938,0.000000e+00,0.181563,0.443588,9.607922
2025_play_2,0.032706,0.002771,0.036451,0.000000e+00,0.422143,1.003838,4.931968,0.015568,0.000603,0.017006,0.000000e+00,0.184164,1.524751,8.214690


In [305]:
# save df to /Users/andrei-macpro/Documents/Data/tracking/features
new_df.to_csv('/Users/andrei-macpro/Documents/Data/tracking/features/play/movement_each.csv')